In [3]:
import duckdb as ddb


In [4]:
ddb.sql("INSTALL httpfs; LOAD httpfs")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [5]:
connection  = ddb.connect("../air_quality.db")

In [6]:
#execute a create schema command
connection.execute("CREATE schema IF NOT EXISTS raw")

In [7]:
connection.sql("""
        SET s3_access_key_id = '';
        SET s3_secret_access_key = '';
        SET s3_region = '';

""")

In [9]:
connection.execute("""
    CREATE TABLE IF NOT EXISTS raw.air_quality_data (
        location_id BIGINT,
        sensors_id BIGINT,
        "location" VARCHAR,
        "datetime" TIMESTAMP,
        lat DOUBLE,
        lon DOUBLE,
        "parameter" VARCHAR,
        units VARCHAR,
        "value" DOUBLE,
        "month" VARCHAR,
        "year" BIGINT,
        ingestion_datetime TIMESTAMP
    );

""")


In [11]:
connection.execute("""
INSERT INTO raw.air_quality_data
SELECT 
    location_id, 
    sensors_id, 
    "location", 
    "datetime", 
    lat, 
    lon, 
    "parameter", 
    units, 
    "value",
    "month", 
    "year",
    current_timestamp AS ingestion_datetime
FROM read_csv('s3://openaq-data-archive/records/csv.gz/locationid=225393/year=2024/month=01/*.csv.gz');
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [12]:
connection.close()